# Lecture 7

In [3]:
# Index assignment for the matrix representation of the game board
player_1 = 1
player_2 = 2
empty = 0

# Game board size
size = 8

In [4]:
def make_game_board(size=8):
    # Make an empty board
    board=[[empty]*size for i in range(size)]
    
    # Even Columns
    for i in range(0,size,2):
        board[1][i]=player_1
        board[-1][i]=player_2
        board[-3][i]=player_2
        
    # Odd Columns
    for i in range(1,size,2):
        board[0][i]=player_1
        board[2][i]=player_1
        board[-2][i]=player_2
    
    return board

In [5]:
left_move=0
right_move=1

player_1_left_move=(1,1)
player_1_right_move=(1,-1)

player_2_left_move=(-1,-1)
player_2_right_move=(-1,1)

moves={ player_1: {left_move: player_1_left_move, 
                   right_move:player_1_right_move},
        player_2: {left_move: player_2_left_move, 
                   right_move: player_2_right_move}}

moves

{1: {0: (1, 1), 1: (1, -1)}, 2: {0: (-1, -1), 1: (-1, 1)}}

In [6]:
def print_message(message,verbose=True):
    if verbose:
        print(message)

In [7]:
def move_piece(board,player,location,move,verbose=True):
    x,y=location
    
    # Check if player's piece is at location
    if not board[x][y] == player:
        print_message("Player does not have piece at location.",verbose)
        return False

    # Fetch the offset for the move
    x_offset,y_offset = moves[player][move]
    
    # Make sure the move is on the board:
    move_possible= x+x_offset < size and \
                    x+x_offset >= 0 and \
                    y+y_offset < size and \
                    y+y_offset >= 0
                
                
    jump_possible= x+2*x_offset < size and \
                    x+2*x_offset >= 0 and \
                    y+2*y_offset < size and \
                    y+2*y_offset >= 0
    
    if not (move_possible or jump_possible):
        print_message("Move is off of board.",verbose)
        return False
        
    # Try the move
    # Is the target space empty
    if move_possible and \
        board[x+x_offset][y+y_offset]==empty:
    
        # Make the move
        # Empty the spot
        board[x][y]=empty
        # Place player in new spot
        board[x+x_offset][y+y_offset]=player
        print_message("Moved.",verbose)            

        return True
    # Does the target space have an opponent's piece, and the space after empty
    elif jump_possible and \
            board[x+x_offset][y+y_offset]!=player and \
            board[x+2*x_offset][y+2*y_offset]==empty:

        # Make the move
        # Empty the spot
        board[x][y]=empty
        # Remove the oppoent's piece
        board[x+x_offset][y+y_offset]=empty
        # Move player to new spot
        board[x+2*x_offset][y+2*y_offset]=player
        print_message("Took opponent's piece.",verbose)
        
        return True
    else:
        print_message("Move not possible.",verbose)
        return False


In [8]:
player_1_piece="X"
player_2_piece="O"
empty_space=" "

In [9]:
space_character= { player_1: player_1_piece,
                    player_2: player_2_piece,
                    empty: empty_space }

space_character

{1: 'X', 2: 'O', 0: ' '}

In [10]:
def draw_board(board):
    for i in range(size):
        for j in range(size):
            print(space_character[board[i][j]],end=" ")
        print()

In [11]:
row_names=list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
row_map=dict(zip(row_names,range(size)))

row_map

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7}

In [13]:
row_names=list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

list(zip(row_names,range(size)))

[('A', 0),
 ('B', 1),
 ('C', 2),
 ('D', 3),
 ('E', 4),
 ('F', 5),
 ('G', 6),
 ('H', 7)]

Note the use of zip, as demonstrated in the previous lecture. Let's do the same for the columns. Here we'll use map to turn a list of numbers to a list of strings: 

In [14]:
column_names=list(map(str,range(1,size+1)))
column_map=dict(zip(column_names,range(size)))

column_map

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}

Now lets draw our new board:

In [15]:
def draw_board(board):
    print(" ",end=" ")
    for j in range(8):
        print(column_names[j],end=" ")
    print()
    
    for i in range(8):
        print(row_names[i],end=" ")
        for j in range(8):
            print(space_character[board[i][j]],end=" ")
        print()

In [16]:
draw_board(make_game_board())

  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C   X   X   X   X 
D                 
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   


Much nicer. 

But now the user will give us locations like "C3" and we'll have to convert it to a pair of indexes for the matrix. These are stored in the `row_map` and `column_map` dictionaries, but we can't trust the user to correctly supply an input, so lets be careful by checking the type of input, length, and making sure it's upper case:

In [17]:
def parse_location(l_string):
    if not isinstance(l_string,str):
        print_message("Bad Input. Location must be string.")
        return False
    
    if len(l_string)!=2:
        print_message("Bad Input. Location must be 2 characters.")
        return False
    
    row=l_string[0].upper()
    col=l_string[1].upper()
    
    if not row in row_names:
        print_message("Bad Row.")
        return False

    if not col in column_names:
        print_message("Bad Column.")
        return False

    return row_map[row],column_map[col]
    

Test our code:

In [18]:
parse_location("C4")

(2, 3)

Similarily setup things so the user can specify "L" or "R" for the moves:

In [19]:
def parse_move(m_string):
    if not isinstance(m_string,str):
        print_message("Bad Input. Location must be string.")
        return -1
    
    if len(m_string)!=1:
        print_message("Bad Input. Location must be 1 character.")
        return -1
    
    if m_string.upper()=="L":
        return left_move

    if m_string.upper()=="R":
        return right_move

    print_message("Bad Move. must be R/L.")
    
    return -1


In [20]:
def nice_move_piece(board,player,location,move):
    loc=parse_location(location)
    mov=parse_move(move)

    if loc and mov!=-1:
        return move_piece(board,player,loc,mov)
    else:
        return print_message("Bad move.")


And finally test again:

In [21]:
board_0=make_game_board()
draw_board(board_0)
nice_move_piece(board_0,player_1,"C4","L")
draw_board(board_0)

  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C   X   X   X   X 
D                 
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C   X       X   X 
D         X       
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   


## A Game Program

Next we don't want the player to be making python calls, the game should show each player the board and ask for input. They should just enter a position and a direction. And the game should keep going until someone wins.

We'll use python's `input` to interact with the player:

In [22]:
def take_move(board,player):
    good_move=False
    
    while not good_move:
        loc_str =input("Input location:")
        if loc_str=="exit":
            return False

        mov_str =input("Input move (L/R):")

        good_move = nice_move_piece(board,player,loc_str,mov_str)
        
    return True


Lets test this function with a single move before moving on (use "C4" and "L"):

In [23]:
board_0=make_game_board()
draw_board(board_0)
take_move(board_0,player_1)
draw_board(board_0)

  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C   X   X   X   X 
D                 
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   
Input location:C2
Input move (L/R):R
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       X   X   X 
D X               
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   


Now lets write a function that determines if the game is won, so it can stop:

In [24]:
def count_pieces(board,player):
    n=0
    for i in range(size):
        for j in range(size):
            if board[i][j]==player:
                n+=1                
    return n


def game_won(board):
    player_1_n=count_pieces(board,player_1)
    player_2_n=count_pieces(board,player_2)

    if player_1_n==0:
        return player_2
    if player_2_n==0:
        return player_1

    return False


Pulling all of these functions together, we get a basic checkers game. Note that it doesn't handle king pieces. And it won't realize when the game is a stale mate. 

In [27]:
def switch_player(player):
    if player==player_1:
        return player_2
    else:
        return player_1    

def checkers_game():
    
    print ("Welcome to Checkers.")
    print ("--------------------")

    # Make a game board
    board_0=make_game_board()
    
    # Start with player 1
    player=player_1
    
    this_game_won=False
    while not this_game_won:
        # Draw the board
        draw_board(board_0)
        
        # Make a move
        print("Player",player,"move:")
        if not take_move(board_0,player):
            break

        # Check if the game has been won
        this_game_won=game_won(board_0)

        # Switch players
        player=switch_player(player)          

    print("Player 1 Pieces:", count_pieces(board_0,player_1))
    print("Player 2 Pieces:", count_pieces(board_0,player_2))
    
    if this_game_won:
        print("Winner is player:",this_game_won)

          

In [28]:
checkers_game()

Welcome to Checkers.
--------------------
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C   X   X   X   X 
D                 
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   
Player 1 move:
Input location:C2
Input move (L/R):L
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       X   X   X 
D     X           
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   
Player 2 move:
Input location:F5
Input move (L/R):L
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       X   X   X 
D     X           
E       O         
F O   O       O   
G   O   O   O   O 
H O   O   O   O   
Player 1 move:
Input location:D3
Input move (L/R):L
Took opponent's piece.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       X   X   X 
D                 
E                 
F O   O   X   O   
G   O   O   O   O 
H O   O   O   O   
Player 2 move:
Input location:exit
Player 1 Pieces: 12
Player 2 Pieces: 11


## Computer Opponent

In [40]:
import copy

def score_board(board,player):
    return count_pieces(board,player)-count_pieces(board,switch_player(player))
    
def generate_moves(board,player,current_player,depth):
    if depth==0:
        return list(),score_board(board,player)
    moves=list()
    scores=list()
    for i in range(size):
        for j in range(size):
            if board[i][j]==current_player:
                for move in [left_move,right_move]:
                    new_board=copy.deepcopy(board)
                    if move_piece(new_board,current_player,(i,j),move,verbose=False):
                        next_moves,score=generate_moves(new_board,player,
                                                        switch_player(current_player),
                                                        depth-1)
                        this_move=[(current_player,(i,j),move)]
                        #this_move.extend(next_moves)
                        moves.append(this_move)
                        scores.append(score)

    return moves,scores
            
def tree_search(t,depth=1):
    if isinstance(t[0],list):
        return sum([tree_search(item,depth+1)/depth for item in t])
    else:
        return max(t)+min(t)
    
def pick_move(board,player,depth=5,func=max):
    moves,scores=generate_moves(board,player,player,depth)
    result=list(map(tree_search,scores))
    move_index=result.index(func(result))
    return moves[move_index][0]
    

In [41]:
board=make_game_board()
possible_moves,scores=generate_moves(board,player_1,player_1,5)

In [42]:
possible_moves

[[(1, (2, 1), 0)],
 [(1, (2, 1), 1)],
 [(1, (2, 3), 0)],
 [(1, (2, 3), 1)],
 [(1, (2, 5), 0)],
 [(1, (2, 5), 1)],
 [(1, (2, 7), 1)]]

In [43]:
scores

[[[[[0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0]],
   [[0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0]],
   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [-1, -1, 0, -1, -1, -1, -1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]],
   [[0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]],
   [[0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0

In [44]:
list(map(tree_search,scores))

[26.999999999999996,
 28.833333333333325,
 26.333333333333332,
 26.333333333333332,
 24.499999999999996,
 27.666666666666664,
 23.666666666666668]

In [45]:
pick_move(board,player_1)

(1, (2, 1), 1)

In [46]:
def checkers_game_AI():
    
    print ("Welcome to Checkers.")
    print ("--------------------")

    # Make a game board
    board_0=make_game_board()
    
    # Start with player 1
    player=player_1
    
    this_game_won=False
    while not this_game_won:
        # Draw the board
        draw_board(board_0)
        
        # Make a move
        if player==player_1:
            print("Player",player,"move:")
            take_move(board_0,player)
        else:
            the_move=pick_move(board_0,player_2)
            #print(the_move)
            move_piece(board_0,*the_move)
            
        # Check if the game has been won
        this_game_won=game_won(board_0)

       # Switch players
        player=switch_player(player)          

    print("Player 1 Pieces:", count_pieces(board_0,player_1))
    print("Player 2 Pieces:", count_pieces(board_0,player_2))
    
    if this_game_won:
        print("Winner is player:",this_game_won)

          

In [ ]:
checkers_game_AI()

Welcome to Checkers.
--------------------
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C   X   X   X   X 
D                 
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   
Player 1 move:
Input location:C3
Input move (L/R):L
Player does not have piece at location.
Input location:C2
Input move (L/R):L
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       X   X   X 
D     X           
E                 
F O   O   O   O   
G   O   O   O   O 
H O   O   O   O   
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       X   X   X 
D     X           
E               O 
F O   O   O       
G   O   O   O   O 
H O   O   O   O   
Player 1 move:
Input location:C4
Input move (L/R):L
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C           X   X 
D     X   X       
E               O 
F O   O   O       
G   O   O   O   O 
H O   O   O   O   
Moved.
  1 2 3 4 5 6 7 8 
A   X   X   X   X 
B X   X   X   X   
C       